In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install -q -U git+https://github.com/sbrugman/SDGym.git@v0.2.2-hw

In [ ]:
!pip install -q -U ../../

In [2]:
from timeit import default_timer as timer
from functools import partial
from random import choices
import logging

In [3]:
import sdgym
from sdgym import load_dataset
from sdgym import benchmark
from sdgym import load_dataset

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

In [5]:
import pgmpy

from pgmpy.models import BayesianModel
from pgmpy.estimators import TreeSearch, HillClimbSearch, BicScore, ExhaustiveSearch, BayesianEstimator
from pgmpy.sampling import BayesianModelSampling

In [6]:
import xgboost as xgb
from xgboost import XGBClassifier

In [7]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.isotonic import IsotonicRegression

In [8]:
from scipy import interpolate

In [9]:
from synthsonic.models.kde_utils import kde_smooth_peaks_1dim, kde_smooth_peaks
from synthsonic.models.kde_copula_nn_pdf import KDECopulaNNPdf

In [10]:
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
logging.basicConfig(level=logging.INFO)

In [12]:
dataset_name = 'credit_numeric'

In [13]:
data, categorical_columns, ordinal_columns = load_dataset(dataset_name)

INFO:sdgym.data:Loading dataset credit (numeric variables, -1 distinct values threshold)
INFO:sdgym.data:Downloading file http://sdgym.s3.amazonaws.com/datasets/credit.npz to /Users/maxbaak/Projects/SDGym_v0.2.2-hw/sdgym/data/credit.npz
INFO:sdgym.data:Downloading file http://sdgym.s3.amazonaws.com/datasets/credit.json to /Users/maxbaak/Projects/SDGym_v0.2.2-hw/sdgym/data/credit.json
INFO:sdgym.data:Columns to zero-code: []


In [14]:
data.shape

(264807, 30)

In [15]:
categorical_columns, ordinal_columns

([], [])

In [16]:
for i in range(data.shape[1]):
    print (i, len(np.unique(data[:, i])))

0 255491
1 256211
2 256139
3 256197
4 256261
5 256151
6 256231
7 256279
8 256251
9 256227
10 256108
11 256202
12 256207
13 256266
14 256195
15 256188
16 256195
17 256249
18 256225
19 256215
20 256218
21 256145
22 256228
23 256177
24 256147
25 256190
26 256316
27 256286
28 31584
29 2


# run sdgym

In [17]:
def KDECopulaNNPdf_Synthesizer(real_data, categorical_columns, ordinal_columns):
    #assert len(categorical_columns + ordinal_columns) <= 1

    all_features = list(range(real_data.shape[1]))
    numerical_features = list(set(all_features) - set(categorical_columns + ordinal_columns))
    data = np.float64(real_data)
    
    n_samples = data.shape[0]
    n_features = data.shape[1]
    
    clf = xgb.XGBClassifier(
        n_estimators=250,
        reg_lambda=1,
        gamma=0,
        max_depth=9
    )
#     clf = MLPClassifier(alpha=0.1, random_state=0, max_iter=1000, early_stopping=True)
    
    kde = KDECopulaNNPdf(
        use_KDE=False, 
        categorical_columns=categorical_columns+ordinal_columns+[29],
        distinct_threshold=-1,
        n_bins=25,
#        clf=clf,
#         n_quantiles=100
#         ordering='mi',
    )
    kde = kde.fit(data)
    
#     X_gen, sample_weight = kde.sample(n_samples)
    X_gen = kde.sample_no_weights(n_samples, show_progress=True, mode='cheap')
    
    X_gen[:, categorical_columns+ordinal_columns] = np.round(X_gen[:, categorical_columns+ordinal_columns])
    X_gen = np.float32(X_gen)
    
    print(X_gen[:, :5])

    return X_gen

In [18]:
def KDECopulaNNPdf_SynthesizerInteger(real_data, categorical_columns, ordinal_columns):
    data = KDECopulaNNPdf_Synthesizer(real_data, categorical_columns, ordinal_columns)
    data = np.round(data)
    
    print(data[:, :5])

    return data

In [19]:
from sdgym.synthesizers import (
    CLBNSynthesizer, CTGANSynthesizer, IdentitySynthesizer, IndependentSynthesizer,
    MedganSynthesizer, PrivBNSynthesizer, TableganSynthesizer, TVAESynthesizer,
    UniformSynthesizer, VEEGANSynthesizer)

all_synthesizers = [
    IdentitySynthesizer,
#    IndependentSynthesizer,
    KDECopulaNNPdf_Synthesizer,
#     KDECopulaNNPdf_SynthesizerInteger,
]

In [20]:
scores = sdgym.run(synthesizers=all_synthesizers, datasets=[dataset_name], iterations=1)

INFO:sdgym.benchmark:0%|          | 0/2 [00:00<?, ?it/s]
INFO:sdgym.benchmark:Evaluating IdentitySynthesizer on dataset credit_numeric; iteration 0; 373.56 MB
INFO:sdgym.data:Loading dataset credit (numeric variables, -1 distinct values threshold)
INFO:sdgym.data:Columns to zero-code: []
INFO:sdgym.benchmark:Running IdentitySynthesizer on dataset credit_numeric; iteration 0; 411.16 MB
INFO:sdgym.synthesizers.base:Fitting IdentitySynthesizer
INFO:sdgym.synthesizers.base:Sampling IdentitySynthesizer
INFO:sdgym.benchmark:Scoring IdentitySynthesizer on dataset credit_numeric; iteration 0; 443.7 MB
INFO:sdgym.evaluate:Evaluating using binary classifier DecisionTreeClassifier
INFO:sdgym.evaluate:Evaluating using binary classifier AdaBoostClassifier
INFO:sdgym.evaluate:Evaluating using binary classifier LogisticRegression
INFO:sdgym.evaluate:Evaluating using binary classifier MLPClassifier
INFO:sdgym.benchmark:Finished IdentitySynthesizer on dataset credit_numeric; iteration 0; 585.24 MB
INFO

Generating for node: 24: 100%|██████████| 30/30 [00:12<00:00,  2.40it/s]
INFO:sdgym.benchmark:Scoring KDECopulaNNPdf_Synthesizer on dataset credit_numeric; iteration 0; 625.74 MB


[[-2.2749374   1.2677987   1.6593778   1.0668854   0.7817724 ]
 [-2.3652058  -0.67815965 -0.5729541  -1.5633943   0.3470698 ]
 [-0.07470031  0.17512068 -0.301692   -1.4472566   0.8928217 ]
 ...
 [-0.03419069 -0.4019206  -0.79529124 -0.5802906   2.4452055 ]
 [-1.5094979   1.3792492  -1.4998144  -2.0975544   1.45936   ]
 [-0.4259839  -0.5671382   0.10612283 -2.6991735  -1.0897039 ]]


INFO:sdgym.evaluate:Evaluating using binary classifier DecisionTreeClassifier
INFO:sdgym.evaluate:Evaluating using binary classifier AdaBoostClassifier
INFO:sdgym.evaluate:Evaluating using binary classifier LogisticRegression
INFO:sdgym.evaluate:Evaluating using binary classifier MLPClassifier
INFO:sdgym.benchmark:Finished KDECopulaNNPdf_Synthesizer on dataset credit_numeric; iteration 0; 772.68 MB
INFO:sdgym.benchmark:100%|##########| 2/2 [08:08<00:00, 244.07s/it]


In [23]:
scores

,credit_numeric/accuracy,credit_numeric/f1,timestamp
VEEGANSynthesizer,NaN,NaN,2020-04-12 09:41:35.096775
CLBNSynthesizer,NaN,NaN,2020-10-17 09:46:54.494331
CTGAN,NaN,NaN,2020-10-17 09:46:54.494331
CTGANSynthesizer,NaN,NaN,2020-10-17 09:46:54.494331
CopulaGAN,NaN,NaN,2020-10-17 09:46:54.494331
GaussianCopulaCategorical,NaN,NaN,2020-10-17 09:46:54.494331
GaussianCopulaCategoricalFuzzy,NaN,NaN,2020-10-17 09:46:54.494331
GaussianCopulaOneHot,NaN,NaN,2020-10-17 09:46:54.494331
IndependentSynthesizer,NaN,NaN,2020-10-17 09:46:54.494331
MedganSynthesizer,NaN,NaN,2020-10-17 09:46:54.494331


In [24]:
scores.tail(4)

,credit_numeric/accuracy,credit_numeric/f1,timestamp
TableganSynthesizer,NaN,NaN,2020-10-17 09:46:54.494331
UniformSynthesizer,NaN,NaN,2020-10-17 09:46:54.494331
IdentitySynthesizer,0.987525,0.532360,2021-05-13 21:09:53.047752
KDECopulaNNPdf_Synthesizer,0.997475,0.181668,2021-05-13 21:09:53.047752
